# 🧠 Divisão da Base, Tratamento de Outliers e Balanceamento (SMOTE)
Este notebook realiza o pré-processamento dos dados da base **PNS 2019**, incluindo:
- Carregamento da base final pré-balanceamento
- Definição da variável alvo e features
- Divisão em treino e teste
- Tratamento de outliers (Capping nos percentis 1% e 99%)
- Codificação (Ordinal, One-Hot e Escalonamento)
- Aplicação do SMOTE para balanceamento do conjunto de treino
- Salvamento dos conjuntos processados em arquivos `.npy`

## 📦 1. Importação das Bibliotecas Necessárias

In [10]:

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler, OrdinalEncoder 
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE
import os
os.environ["LOKY_MAX_CPU_COUNT"] = "10" 


## 📂 2. Carregar a Base de Dados Final

In [11]:

arquivo_para_ler = 'pns_final_pre_balanceamento.csv'
try:
    df = pd.read_csv(arquivo_para_ler, sep=';', decimal=',')
    print(f"Arquivo '{arquivo_para_ler}' carregado com sucesso.")
    print(f"Shape da base (desbalanceada): {df.shape}")
except FileNotFoundError:
    print(f"🛑 Erro: Arquivo '{arquivo_para_ler}' não foi encontrado.")
    exit()


Arquivo 'pns_final_pre_balanceamento.csv' carregado com sucesso.
Shape da base (desbalanceada): (1288, 31)


## 🎯 3. Definir a Variável Alvo (y)

In [12]:

TARGET_COLUMN = 'DIABETES' 

if TARGET_COLUMN not in df.columns:
    print(f"🛑 Erro: A coluna alvo '{TARGET_COLUMN}' não foi encontrada no DataFrame.")
    exit()


## 🧩 4. Definir Listas de Features (X)

In [13]:

colunas_numericas = [
    "IDADE", "RENDA_TOTAL", "Peso_Final", "Altura_Final_cm", "IMC"
]

colunas_ordinais = [
    "FEIJAO", "VERDURA_LEGUME", "FREQ_VERDURA_LEGUME", "CARNE_VERMELHA", 
    "FRANGO_GALINHA", "PEIXE", "SUCO_INDUSTRIALIZADO", "SUCO_NATURAL", 
    "FRUTA_SEMANA", "FREQ_FRUTA_DIA", "REFRIGERANTE_SEMANA", "DOCES_SEMANA", 
    "SUBSTITUIR_REFEICAO_DOCE_SEMANA", "CONSUMO_SAL", "NIVEL_CONSUMO_ALCOOL", 
    "NIVEL_ATIVIDADE_FISICA", "FAIXA_RENDA_SM"
]

colunas_nominais = [
    "SEXO", "PLANO_SAUDE", "GRAVIDEZ", "TIPO_SUCO_INDUSTIALIZADO", 
    "TIPO_REFRIGERANTE", "LEITE_SEMANA", "TIPO_LEITE", "COMA_DIABETICO"
]


## ✂️ 5. Dividir a Base em Treino e Teste

In [14]:

X = df.drop(columns=[TARGET_COLUMN])
y = df[TARGET_COLUMN]

X_train, X_test, y_train, y_test = train_test_split(
    X, 
    y, 
    test_size=0.3,
    random_state=42,
    stratify=y
)

print("\n--- Distribuição das classes (Antes do SMOTE) ---")
print("Treino (y_train):\n", y_train.value_counts(normalize=True))
print("\nTeste (y_test):\n", y_test.value_counts(normalize=True))



--- Distribuição das classes (Antes do SMOTE) ---
Treino (y_train):
 DIABETES
2    0.732519
1    0.267481
Name: proportion, dtype: float64

Teste (y_test):
 DIABETES
2    0.73385
1    0.26615
Name: proportion, dtype: float64


## 📊 6. Tratamento de Outliers (Capping nos Percentis 1% e 99%)

In [15]:

print("\nAplicando Capping (Tratamento de Outliers) nos Percentis 1% e 99%...")

for col in colunas_numericas:
    limite_inferior = X_train[col].quantile(0.01)
    limite_superior = X_train[col].quantile(0.99)
    
    print(f"  [{col}]: Limites (de Treino) -> Inf={limite_inferior:.2f}, Sup={limite_superior:.2f}")
    
    X_train[col] = X_train[col].clip(lower=limite_inferior, upper=limite_superior)
    X_test[col] = X_test[col].clip(lower=limite_inferior, upper=limite_superior)

print("Capping concluído.")



Aplicando Capping (Tratamento de Outliers) nos Percentis 1% e 99%...
  [IDADE]: Limites (de Treino) -> Inf=35.00, Sup=78.00
  [RENDA_TOTAL]: Limites (de Treino) -> Inf=100.00, Sup=13300.00
  [Peso_Final]: Limites (de Treino) -> Inf=45.20, Sup=119.40
  [Altura_Final_cm]: Limites (de Treino) -> Inf=145.00, Sup=188.00
  [IMC]: Limites (de Treino) -> Inf=18.44, Sup=41.80
Capping concluído.


## ⚙️ 7. Criação dos Pipelines de Pré-processamento

In [16]:

numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])

ordinal_transformer = Pipeline(steps=[
    ('ordinal', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1))
])

nominal_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, colunas_numericas),
        ('nom', nominal_transformer, colunas_nominais),
        ('ord', ordinal_transformer, colunas_ordinais) 
    ],
    remainder='drop'
)

print("\nAplicando One-Hot, Ordinal Encoding e Scaling...")

X_train_final = preprocessor.fit_transform(X_train)
X_test_final = preprocessor.transform(X_test)

print("Pré-processamento concluído.")



Aplicando One-Hot, Ordinal Encoding e Scaling...
Pré-processamento concluído.


## ⚖️ 8. Aplicar o SMOTE (Balanceamento do Conjunto de Treino)

In [17]:

print("\nAplicando SMOTE apenas nos dados de TREINO...")

smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train_final, y_train)

print("\n--- Distribuição das classes (Depois do SMOTE) ---")
print("Treino (y_train_smote):\n", pd.Series(y_train_smote).value_counts(normalize=True))

print("\n--- SHAPES FINAIS ---")
print(f"X_train_smote (p/ Modelo): {X_train_smote.shape}")
print(f"y_train_smote (p/ Modelo): {y_train_smote.shape}")
print(f"X_test_final (p/ Avaliação): {X_test_final.shape}")
print(f"y_test (p/ Avaliação): {y_test.shape}")

print("\n✅ Processo concluído!")



Aplicando SMOTE apenas nos dados de TREINO...

--- Distribuição das classes (Depois do SMOTE) ---
Treino (y_train_smote):
 DIABETES
2    0.5
1    0.5
Name: proportion, dtype: float64

--- SHAPES FINAIS ---
X_train_smote (p/ Modelo): (1320, 50)
y_train_smote (p/ Modelo): (1320,)
X_test_final (p/ Avaliação): (387, 50)
y_test (p/ Avaliação): (387,)

✅ Processo concluído!


## 💾 9. Salvar os Conjuntos Processados em Arquivos `.npy`

In [18]:

np.save('X_train_smote.npy', X_train_smote)
np.save('y_train_smote.npy', y_train_smote)
np.save('X_test_final.npy', X_test_final)
np.save('y_test.npy', y_test)

print("\n💾 Conjuntos de treino (balanceado) e teste (desbalanceado) salvos como arquivos .npy.")



💾 Conjuntos de treino (balanceado) e teste (desbalanceado) salvos como arquivos .npy.
